One model always consumed one unit of gpu. For now we do not support distributed batch processing to multiple GPUs from one model. But we can initiate multiple models to multiple GPUs.

model_emotion -> GPU0

model_sentiment -> GPU1

model_translation -> GPU2

and so on.

In [ ]:
!git pull

In [1]:
%%time

import malaya

CPU times: user 5.94 s, sys: 2.35 s, total: 8.29 s
Wall time: 4.01 s


In [2]:
!nvidia-smi

Tue Jul  7 21:32:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.129      Driver Version: 410.129      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0  On |                    0 |
| N/A   55C    P0    42W / 300W |      0MiB / 32475MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
| N/A   65C    P0   250W / 300W |  31452MiB / 32478MiB |     89%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

Right now all the GPUs in resting mode, no computation happened.

## GPU Rules

1. By default, all models will initiate in first GPU, unless override `gpu` parameter in any load model API. Example as below.
2. Malaya will not consumed all available GPU memory, but slowly grow based on batch size. This growth only towards positive (use more GPU memory) dynamically, but will not reduce GPU memory if feed small batch size.
3. Use `malaya.clear_session` to clear session from unused models but this will not free GPU memory.
4. By default Malaya will not set max cap for GPU memory, to put a cap, override `gpu_limit` parameter in any load model API. `gpu_limit` should 0 < `gpu_limit` < 1. If `gpu_limit = 0.3`, it means the model will not use more than 30% of GPU memory.

In [3]:
anger_text = 'babi la company ni, aku dah la penat datang dari jauh'
fear_text = 'takut doh tengok cerita hantu tadi'
happy_text = 'bestnya dapat tidur harini, tak payah pergi kerja'
love_text = 'aku sayang sgt dia dah doh'
sadness_text = 'kecewa tengok kerajaan baru ni, janji ape pun tak dapat'
surprise_text = 'sakit jantung aku, terkejut dengan cerita hantu tadi'

In [4]:
model = malaya.emotion.transformer(model = 'bert', gpu = '0')

In [5]:
%%time

model.predict_proba(
    [anger_text, fear_text, happy_text, love_text, sadness_text, surprise_text]
)

CPU times: user 1.94 s, sys: 541 ms, total: 2.48 s
Wall time: 2.52 s


[{'anger': 0.9998965,
  'fear': 1.7692768e-05,
  'happy': 1.8747674e-05,
  'love': 1.656881e-05,
  'sadness': 3.130815e-05,
  'surprise': 1.9183277e-05},
 {'anger': 7.4469484e-05,
  'fear': 0.99977416,
  'happy': 6.824215e-05,
  'love': 2.773282e-05,
  'sadness': 1.9767067e-05,
  'surprise': 3.5663204e-05},
 {'anger': 0.99963737,
  'fear': 3.931449e-05,
  'happy': 0.0001562279,
  'love': 3.3580774e-05,
  'sadness': 0.00011328616,
  'surprise': 2.0134145e-05},
 {'anger': 3.1319763e-05,
  'fear': 1.7286226e-05,
  'happy': 2.9899325e-05,
  'love': 0.99987257,
  'sadness': 2.7867774e-05,
  'surprise': 2.096328e-05},
 {'anger': 8.965934e-05,
  'fear': 1.8196944e-05,
  'happy': 2.9275663e-05,
  'love': 1.7211949e-05,
  'sadness': 0.9998247,
  'surprise': 2.0944033e-05},
 {'anger': 4.132152e-05,
  'fear': 6.202527e-05,
  'happy': 3.1012056e-05,
  'love': 5.3896296e-05,
  'sadness': 6.202101e-05,
  'surprise': 0.9997497}]

In [8]:
!nvidia-smi

Tue Jul  7 21:32:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.129      Driver Version: 410.129      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0  On |                    0 |
| N/A   56C    P0    58W / 300W |   1099MiB / 32475MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
| N/A   64C    P0   219W / 300W |  31452MiB / 32478MiB |     99%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [7]:
malaya.clear_session(model)

True